# Statistical Inference with Bootstrap and SE formula

In this notebook we're going use statistical inference to make a decision about which business chain to invest in, looking at 2 different methods of making an inference to the underlying population from sample data. 

In [1]:
from google.oauth2 import service_account
from google.cloud import bigquery
import configparser
import pandas as pd
import numpy as np
import random
from collections import Counter
pd.options.display.float_format = '{:,.4f}'.format

In [2]:
KEY_PATH = "/mnt/c/Users/Ron/git-repos/yelp-data/gourmanddwh-f75384f95e86.json"
CREDS = service_account.Credentials.from_service_account_file(KEY_PATH)
client = bigquery.Client(credentials=CREDS, project=CREDS.project_id)



To begin we can run some sql scripts to get the data from our DWH in BigQuery.Then we'll serialize it as a parquet file in the event that we want to use the data again.

In [ ]:
cg_file = open('sql_scripts/county_growth_est.sql','r')
county_growth_query =  cg_file.read()
cg_dataframe: pd.DataFrame = (
    client.query(county_growth_query)
    .result()
    .to_dataframe()
)
cg_dataframe.to_parquet('cg_est', 'pyarrow','snappy', partition_cols=['StateName'])

In [ ]:
holding_file = open('sql_scripts/business_daily_holding.sql')
holding_query = holding_file.read()

holding_dataframe : pd.DataFrame = (
    client.query(holding_query)
    .result()
    .to_dataframe()
)
holding_dataframe.to_parquet('bus_holdings', 'pyarrow','snappy', partition_cols=['CloseDate'])

In [ ]:
bus_cat_file = open('sql_scripts/business_category_location.sql')
bus_cat_query = bus_cat_file.read()

bus_cat_dataframe: pd.DataFrame = (
    client.query(bus_cat_query)
    .result()
    .to_dataframe()
)

bus_cat_dataframe.to_parquet('bus_cats', 'pyarrow','snappy', partition_cols=['StateName'])

In [4]:
cg_df = pd.read_parquet('cg_est', engine='pyarrow')
bh_df: pd.DataFrame = pd.read_parquet('bus_holdings', engine='pyarrow')

In [6]:
holdings_count_df = bh_df.groupby(['CloseDate'], as_index=False)['BusinessName'].count()
holdings_count_df.sort_values('BusinessName', ascending=False).head()

,CloseDate,BusinessName
0,2021-12-29,63317
3,2022-01-07,62551
4,2022-01-08,62545
5,2022-01-09,62538
7,2022-01-11,62504


The question we'll be looking to answer is whether one business chain makes for a better investment as opposed to another. While we have a pretty good selection of businesses to choose some any analysis will alaways be facilitated with a higher sample size. \
For this purpose we'll look to compare the 2 businesses with the highest number of observations.
> Note: In this case an observation corresponds to the statistics we have on a given business chain's physical store location on a given day

As we can see below the chains with number of observations are **Subway** and **Mcdonald's**.

In [7]:
bus_count_chn_df = bh_df.groupby(['ChainName'], as_index=False)['BusinessName'].count()
bus_count_chn_df.sort_values('BusinessName', ascending=False).head(10)

,ChainName,BusinessName
40277,Subway,24708
27308,McDonald's,21431
33368,Pizza Hut,13506
41179,Taco Bell,10222
39094,Sonic Drive-In,10077
6342,Burger King,9581
12250,Domino's Pizza,7081
48554,Wendy's,6813
39789,Starbucks,6663
11236,Dairy Queen,6617


***continue with explaining regex of mcdonalds***

In [8]:
bh_df.ChainName.str.lower().str.contains(r'mcdonald[\']?s').sum()
# 3 different variations
bh_df.loc[bh_df.ChainName.str.lower().str.contains(r'mcdonald[\']?s'), 'ChainName'].value_counts()

McDonald's    21431
Mcdonald's       33
McDonalds        30
Name: ChainName, dtype: int64

In [9]:
mcdonalds = bh_df.loc[bh_df.ChainName.str.lower().str.contains(r'mcdonald[\']?s'), :].reset_index(drop=True)
mcdonalds.head()

,BusinessName,ChainName,BusinessRating,ReviewCount,previous_review_cnt,previous_rating,abs_review_diff,abs_rating_diff,total_review_cnt_delta,total_bus_rating_delta,CloseDate
0,mcdonalds-ada-2,McDonald's,2.000000000,19,NaN,None,NaN,None,-1.0000,0E-9,2021-12-29
1,mcdonalds-albert-lea,McDonald's,3.000000000,4,NaN,None,NaN,None,0.0000,0E-9,2021-12-29
2,mcdonalds-albert-lea-2,McDonald's,3.500000000,3,NaN,None,NaN,None,0.0000,0E-9,2021-12-29
3,mcdonalds-albert-lea-3,McDonald's,1.000000000,3,NaN,None,NaN,None,0.0000,0E-9,2021-12-29
4,mcdonalds-algona,McDonald's,3.000000000,2,NaN,None,NaN,None,1.0000,-0.500000000,2021-12-29


In [10]:
mcdonalds.shape

(21494, 11)

In [11]:
subway = bh_df.loc[bh_df.ChainName == 'Subway'].reset_index(drop=True)
subway.head()

,BusinessName,ChainName,BusinessRating,ReviewCount,previous_review_cnt,previous_rating,abs_review_diff,abs_rating_diff,total_review_cnt_delta,total_bus_rating_delta,CloseDate
0,subway-abbeville-10,Subway,2.000000000,2,NaN,None,NaN,None,0.0000,0E-9,2021-12-29
1,subway-adrian-6,Subway,2.000000000,1,NaN,None,NaN,None,0.0000,0E-9,2021-12-29
2,subway-albany-83,Subway,3.000000000,2,NaN,None,NaN,None,0.0000,0E-9,2021-12-29
3,subway-albert-lea-6,Subway,2.500000000,4,NaN,None,NaN,None,-1.0000,-1.000000000,2021-12-29
4,subway-albert-lea-7,Subway,2.500000000,2,NaN,None,NaN,None,0.0000,0E-9,2021-12-29


In [12]:
subway.columns

Index(['BusinessName', 'ChainName', 'BusinessRating', 'ReviewCount',
       'previous_review_cnt', 'previous_rating', 'abs_review_diff',
       'abs_rating_diff', 'total_review_cnt_delta', 'total_bus_rating_delta',
       'CloseDate'],
      dtype='object')

In [13]:
ordered_subway = subway.sort_values(['BusinessName', 'CloseDate'], ascending=True).reset_index(drop=True)
ordered_subway

,BusinessName,ChainName,BusinessRating,ReviewCount,previous_review_cnt,previous_rating,abs_review_diff,abs_rating_diff,total_review_cnt_delta,total_bus_rating_delta,CloseDate
0,subway-abbeville-10,Subway,2.000000000,2,NaN,None,NaN,None,0.0000,0E-9,2021-12-29
1,subway-abbeville-10,Subway,2.000000000,2,2.0000,2.000000000,0.0000,0E-9,0.0000,0E-9,2022-01-07
2,subway-abbeville-10,Subway,2.000000000,2,2.0000,2.000000000,0.0000,0E-9,0.0000,0E-9,2022-01-08
3,subway-abbeville-10,Subway,2.000000000,2,2.0000,2.000000000,0.0000,0E-9,0.0000,0E-9,2022-01-09
4,subway-abbeville-10,Subway,2.000000000,2,2.0000,2.000000000,0.0000,0E-9,0.0000,0E-9,2022-01-10
...,...,...,...,...,...,...,...,...,...,...,...
24703,subway-zapata-3,Subway,4.000000000,2,2.0000,4.000000000,0.0000,0E-9,1.0000,-1.000000000,2022-02-03
24704,subway-zapata-3,Subway,4.000000000,2,2.0000,4.000000000,0.0000,0E-9,1.0000,-1.000000000,2022-02-04
24705,subway-zapata-3,Subway,4.000000000,2,2.0000,4.000000000,0.0000,0E-9,1.0000,-1.000000000,2022-02-05
24706,subway-zapata-3,Subway,4.000000000,2,2.0000,4.000000000,0.0000,0E-9,1.0000,-1.000000000,2022-02-06


In [14]:
ordered_mcdonalds = mcdonalds.sort_values(['BusinessName', 'CloseDate'], ascending=True).reset_index(drop=True)
ordered_mcdonalds

,BusinessName,ChainName,BusinessRating,ReviewCount,previous_review_cnt,previous_rating,abs_review_diff,abs_rating_diff,total_review_cnt_delta,total_bus_rating_delta,CloseDate
0,mcdonalds-ada-2,McDonald's,2.000000000,19,NaN,None,NaN,None,-1.0000,0E-9,2021-12-29
1,mcdonalds-ada-2,McDonald's,2.000000000,19,19.0000,2.000000000,0.0000,0E-9,-1.0000,0E-9,2022-01-07
2,mcdonalds-ada-2,McDonald's,2.000000000,18,19.0000,2.000000000,-1.0000,0E-9,-1.0000,0E-9,2022-01-08
3,mcdonalds-ada-2,McDonald's,2.000000000,18,18.0000,2.000000000,0.0000,0E-9,-1.0000,0E-9,2022-01-09
4,mcdonalds-ada-2,McDonald's,2.000000000,18,18.0000,2.000000000,0.0000,0E-9,-1.0000,0E-9,2022-01-10
...,...,...,...,...,...,...,...,...,...,...,...
21489,mcdonalds-zapata,McDonald's,2.000000000,5,5.0000,2.000000000,0.0000,0E-9,0.0000,0E-9,2022-02-03
21490,mcdonalds-zapata,McDonald's,2.000000000,5,5.0000,2.000000000,0.0000,0E-9,0.0000,0E-9,2022-02-04
21491,mcdonalds-zapata,McDonald's,2.000000000,5,5.0000,2.000000000,0.0000,0E-9,0.0000,0E-9,2022-02-05
21492,mcdonalds-zapata,McDonald's,2.000000000,5,5.0000,2.000000000,0.0000,0E-9,0.0000,0E-9,2022-02-06


In [18]:
ordered_subway.loc[:, 'review_count_abs_diff_lag_1'] = (ordered_subway.ReviewCount - ordered_subway.previous_review_cnt)
ordered_subway.loc[:, 'review_count_perc_diff_lag_1'] = (ordered_subway['review_count_abs_diff_lag_1'] / ordered_subway.previous_review_cnt) ** 100
ordered_subway.sort_values('review_count_perc_diff_lag_1', ascending=False)
ordered_subway['review_count_perc_diff_lag_1'].value_counts(sort=True)

0.0000    23781
0.0000       18
0.0000       17
0.0000       16
0.0000       12
0.0000       11
1.0000        7
0.0000        5
0.0000        4
0.0000        3
0.0000        2
0.0000        2
0.0000        1
0.0000        1
0.0000        1
0.0000        1
0.0000        1
Name: review_count_perc_diff_lag_1, dtype: int64

In [19]:
ordered_subway['review_count_perc_diff_lag_1'].value_counts(bins=10)

(-0.002, 0.1]    23876
(0.9, 1.0]           7
(0.1, 0.2]           0
(0.2, 0.3]           0
(0.3, 0.4]           0
(0.4, 0.5]           0
(0.5, 0.6]           0
(0.6, 0.7]           0
(0.7, 0.8]           0
(0.8, 0.9]           0
Name: review_count_perc_diff_lag_1, dtype: int64

In [20]:
ordered_subway['review_count_abs_diff_lag_1'].value_counts(sort=True)

0.0000     23814
1.0000        71
-1.0000       30
2.0000         1
Name: review_count_abs_diff_lag_1, dtype: int64

In [21]:
ordered_mcdonalds.loc[:, 'review_count_abs_diff_lag_1'] = (ordered_mcdonalds.ReviewCount - ordered_mcdonalds.previous_review_cnt)
ordered_mcdonalds.loc[:, 'review_count_perc_diff_lag_1'] = (ordered_mcdonalds['review_count_abs_diff_lag_1'] / ordered_mcdonalds.previous_review_cnt) ** 100
ordered_mcdonalds['review_count_abs_diff_lag_1'].value_counts(sort=True)

0.0000     20435
1.0000       257
-1.0000       86
2.0000        24
3.0000         3
-2.0000        2
Name: review_count_abs_diff_lag_1, dtype: int64

In [22]:
ordered_subway_sin_na = ordered_subway.dropna(subset=['review_count_abs_diff_lag_1'])
ordered_subway_sin_na.loc[:, 'review_count_abs_diff_lag_1_gte_1'] = ordered_subway_sin_na['review_count_abs_diff_lag_1'] >= 1
subway_daily_rc_diff_mean = ordered_subway_sin_na['review_count_abs_diff_lag_1_gte_1'].mean(skipna=True)
subway_daily_rc_diff_mean

/mnt/c/Users/Ron/git-repos/data-analysis-business-economics-policy/da_venv/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


0.0030105368790767687

In [23]:
ordered_mcdonalds_sin_na = ordered_mcdonalds.dropna(subset=['review_count_abs_diff_lag_1'])
ordered_mcdonalds_sin_na.loc[:, 'review_count_abs_diff_lag_1_gte_1'] = ordered_mcdonalds_sin_na['review_count_abs_diff_lag_1'] >= 1
mcdonalds_daily_rc_diff_mean = ordered_mcdonalds_sin_na['review_count_abs_diff_lag_1_gte_1'].mean(skipna=True)
mcdonalds_daily_rc_diff_mean

0.01364925265535637

In [24]:
sample_means_subway = []
sample_means_mcdonalds = []
# random.choices allow us to do it with replacement
for i in range(10):
    ixs = random.choices(population = ordered_mcdonalds_sin_na.index.tolist() ,k=ordered_mcdonalds_sin_na.shape[0])
    count_dict = Counter(ixs)
    print(count_dict.most_common(1))

[(14009, 6)]
[(2950, 7)]
[(9507, 8)]
[(10576, 6)]
[(11391, 8)]
[(209, 6)]
[(5856, 7)]
[(3602, 7)]
[(13161, 6)]
[(974, 7)]


In [25]:
sample_means_subway = []
sample_means_mcdonalds = []
#rn generator
random.seed(42)

for i in range(10000):
    #seems this way would be less efficient but nonetheless viable
    # mcd_ixs = random.choices(population = ordered_mcdonalds_sin_na.index.tolist() ,k=ordered_mcdonalds_sin_na.shape[0])
    # mcd_sample_mean = ordered_mcdonalds_sin_na.loc[mcd_ixs,'review_count_abs_diff_lag_1_gte_1'].mean()
    # sample_means_mcdonalds.append(mcd_sample_mean)

    # subway_ixs = random.choices(population = ordered_subway_sin_na.index.tolist() ,k=ordered_subway_sin_na.shape[0])
    # subway_sample_mean = ordered_subway_sin_na.loc[subway_ixs,'review_count_abs_diff_lag_1_gte_1'].mean()
    # sample_means_subway.append(subway_sample_mean)

    mcd_ixs = random.choices(population = ordered_mcdonalds_sin_na.review_count_abs_diff_lag_1_gte_1.tolist() ,k=ordered_mcdonalds_sin_na.shape[0])
    mcd_sample_mean = np.mean(mcd_ixs)
    sample_means_mcdonalds.append(mcd_sample_mean)

    subway_ixs = random.choices(population = ordered_subway_sin_na.review_count_abs_diff_lag_1_gte_1.tolist() ,k=ordered_subway_sin_na.shape[0])
    subway_sample_mean = np.mean(subway_ixs)
    sample_means_subway.append(subway_sample_mean)

In [26]:
sample_means_subway_mean = np.mean(sample_means_subway)

sample_means_mcdonalds_mean = np.mean(sample_means_mcdonalds)

In [27]:
subway_se = np.std(sample_means_subway)
subway_se 

0.00035164206546019243

In [28]:
mcdonalds_se = np.std(sample_means_mcdonalds)
mcdonalds_se

0.0008058193797289551

In [29]:
print(f'Subway CI with 95% confidence {subway_daily_rc_diff_mean - (subway_se * 2):.5f} <---> {subway_daily_rc_diff_mean + (subway_se * 2):.5f} with ', end=' ')
print(f'and bootstrap se estimate of {subway_se:.5f}')

Subway CI with 95% confidence 0.00231 <---> 0.00371 with  and bootstrap se estimate of 0.00035


In [30]:
print(f'Mcdonald\'s CI with 95% confidence {mcdonalds_daily_rc_diff_mean - (mcdonalds_se * 2):.5f} <---> {mcdonalds_daily_rc_diff_mean + (mcdonalds_se * 2):.5f} with', end=' ')
print(f'and bootstrap se estimate of {mcdonalds_se:.5f}')

Mcdonald's CI with 95% confidence 0.01204 <---> 0.01526 with and bootstrap se estimate of 0.00081


In [31]:
subway_formula_se = (1 / np.sqrt(ordered_subway_sin_na.shape[0])) * np.std(ordered_subway_sin_na.review_count_abs_diff_lag_1_gte_1)
subway_formula_se

0.00035426070916424687

In [32]:
mcdonalds_formula_se = (1 / np.sqrt(ordered_mcdonalds_sin_na.shape[0])) * np.std(ordered_mcdonalds_sin_na.review_count_abs_diff_lag_1_gte_1)
mcdonalds_formula_se

0.0008043876457649798

In [33]:
print(f'Subway CI with 95% confidence {subway_daily_rc_diff_mean - (subway_formula_se * 2):.5f} <---> {subway_daily_rc_diff_mean + (subway_formula_se * 2):.5f} with ', end=' ')
print(f'and se formula estimate of {subway_formula_se:.5f}')

Subway CI with 95% confidence 0.00230 <---> 0.00372 with  and se formula estimate of 0.00035


In [34]:
print(f'Mcdonald\'s CI with 95% confidence {mcdonalds_daily_rc_diff_mean - (mcdonalds_formula_se * 2):.5f} <---> {mcdonalds_daily_rc_diff_mean + (mcdonalds_formula_se * 2):.5f} with', end=' ')
print(f'and se formula estimate of {mcdonalds_formula_se:.5f}')

Mcdonald's CI with 95% confidence 0.01204 <---> 0.01526 with and se formula estimate of 0.00080


# Main Takeaway